Resultado é dado por RMSLE - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.root_mean_squared_log_error.html#sklearn.metrics.root_mean_squared_log_error

O pior score atualmente está em 4.37970, o melhor em 0.37768, com 671 participantes nesse momento (30/05)

# Desafio Store Sales

# A fazer

- União treino/teste para feature selection
- Feature selection (NAs, correlação, multicolineariedade, CV)
- Separação treino/teste e variáveis categóricas/numéricas
- Dropar as colunas 'Id' antes de fazer o modelo
- De repente primeiro prever o número de transações por loja, depois prever o valor de vendas?

## Carregamento dos dados

Importação das bibliotecas necessárias à seção

In [1]:
import pandas as pd
import numpy as np
import pyarrow

pd.set_option('display.max_columns', None)
np.set_printoptions(suppress=True, precision=6)

## CSV, ajustes e parquet

### Importação dos datasets CSV e ajuste dos tipos das variáveis

In [8]:
"""
treino = pd.read_csv("train.csv")
treino["date"] = pd.to_datetime(treino["date"])

teste = pd.read_csv("test.csv")
teste["date"] = pd.to_datetime(teste["date"])

feriados = pd.read_csv("holidays_events.csv")
feriados["date"] = pd.to_datetime(feriados["date"])

gasolina = pd.read_csv("oil.csv")
gasolina["date"] = pd.to_datetime(gasolina["date"])

transacoes = pd.read_csv("transactions.csv")
transacoes["date"] = pd.to_datetime(transacoes["date"])

lojas = pd.read_csv("stores.csv") # não tem data envolvida. Dropa city/state?

# Transformação das demais variáveis categóricas efetivamente em categorias
treino["store_nbr"] = treino["store_nbr"].astype("category")
treino["family"] = treino["family"].astype("category")

teste["store_nbr"] = teste["store_nbr"].astype("category")
teste["family"] = teste["family"].astype("category")

feriados["type"] = feriados["type"].astype("category")
feriados["locale"] = feriados["locale"].astype("category")
feriados["locale_name"] = feriados["locale_name"].astype("category")
feriados["transferred"] = feriados["transferred"].astype("category")
feriados = feriados[feriados["transferred"] == False]
feriados.drop(feriados[feriados["date"] < "2013-01-01"].index, axis = 0, inplace = True)
feriados.drop(feriados[feriados["date"] >= "2017-09-01"].index, axis = 0, inplace = True)
feriados.reset_index(inplace = True)
feriados.drop("index", axis = 1, inplace = True)


gasolina.fillna(method='bfill', inplace=True)

transacoes["store_nbr"] = transacoes["store_nbr"].astype("category")

lojas["store_nbr"] = lojas["store_nbr"].astype("category")
lojas["type"] = lojas["type"].astype("category")
lojas["cluster"] = lojas["cluster"].astype("category")

# Exportação para parquet dos dados já brevemente ajustados 

teste.to_parquet("teste.parquet")
treino.to_parquet("treino.parquet")
feriados.to_parquet("feriados.parquet")
gasolina.to_parquet("gasolina.parquet")
transacoes.to_parquet("transacoes.parquet")
lojas.to_parquet("lojas.parquet")
"""

#### Feature Engeneering

In [ ]:
"""
###
# Dataset de treino
### 

# store_type
for i in range(len(treino["store_type"])):
    treino["store_type"].iloc[i] = lojas["type"][treino["store_nbr"].iloc[i] == lojas["store_nbr"]].values[0]

# cluster
for i in range(len(treino["cluster"])):
    treino["cluster"].iloc[i] = lojas["cluster"][treino["store_nbr"].iloc[i] == lojas["store_nbr"]].values[0]

# holidays_events
for i in range(len(feriados["date"])):
    feriado = feriados["date"].iloc[i] == treino["date"]
    treino["feriado"][feriado] = 1

treino["feriado"] = treino["feriado"].astype("category")

# oil
for i in range(len(gasolina["date"])):
    mask = gasolina["date"].iloc[i] == treino["date"]
    treino["gasolina"][mask] = gasolina["dcoilwtico"].loc[i]

# transactions
for date in transacoes["date"].unique():
    mask = transacoes[transacoes["date"] == date]
    for i in mask["store_nbr"]:
        if mask.loc[(mask["store_nbr"] == i), "transactions"].values[0] == 0: pass
        else:
            mask_venda = treino[(treino["date"] == date) & (treino["store_nbr"] == i)]
            vendas = mask_venda["sales"].sum()
            trans_loja = mask[mask["store_nbr"] == i]["transactions"]
            ticket_medio = vendas/trans_loja.values[0]
            filtro = treino[(treino["date"] == date) & (treino["store_nbr"] == i) & (treino["sales"] > 0)].index
            treino.loc[filtro, "ticket_medio"] = ticket_medio


            
###
# Dataset de teste
### 

# store_type
for i in range(len(teste["store_type"])):
    teste["store_type"].iloc[i] = lojas["type"][teste["store_nbr"].iloc[i] == lojas["store_nbr"]].values[0]

# cluster
for i in range(len(teste["cluster"])):
    teste["cluster"].iloc[i] = lojas["cluster"][teste["store_nbr"].iloc[i] == lojas["store_nbr"]].values[0]

# holidays_events
for i in range(len(feriados["date"])):
    feriado = feriados["date"].iloc[i] == teste["date"]
    teste["feriado"][feriado] = 1

teste["feriado"] = teste["feriado"].astype("category")

# oil
for i in range(len(gasolina["date"])):
    mask = gasolina["date"].iloc[i] == teste["date"]
    teste["gasolina"][mask] = gasolina["dcoilwtico"].loc[i]


    
###
# Limpeza do workspace
### 
del (mask, feriado, filtro, ticket_medio, trans_loja, vendas, mask_venda, copia, data_teste, date, i, j)

"""

### Carregamento dos dados (parquet)

In [240]:
treino = pd.read_parquet("treino.parquet")
teste = pd.read_parquet("teste.parquet")
#feriados = pd.read_parquet("feriados.parquet")
#gasolina = pd.read_parquet("gasolina.parquet")
#transacoes = pd.read_parquet("transacoes.parquet")
#lojas = pd.read_parquet("lojas.parquet")

## Feature Selection

In [242]:
treino

,id,date,store_nbr,family,sales,onpromotion,store_type,cluster,feriado,gasolina,ticket_medio
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,D,13,1,93.14,0.000000
1,1,2013-01-01,1,BABY CARE,0.000,0,D,13,1,93.14,0.000000
2,2,2013-01-01,1,BEAUTY,0.000,0,D,13,1,93.14,0.000000
3,3,2013-01-01,1,BEVERAGES,0.000,0,D,13,1,93.14,0.000000
4,4,2013-01-01,1,BOOKS,0.000,0,D,13,1,93.14,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0,B,6,1,47.57,8.648017
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,B,6,1,47.57,8.648017
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148,B,6,1,47.57,8.648017
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,B,6,1,47.57,8.648017


In [247]:
treino.to_parquet("treino.parquet")